In [1]:
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import random
import os
from pprint import pprint
os.environ["CUDA_VISIBLE_DEVICES"] = '4'
import tensorflow as tf
random.seed(84)
np.random.seed(58)
tf.random.set_seed(34)
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard, LearningRateScheduler, EarlyStopping
from tensorflow.keras import backend as K
import tensorflow_datasets as tfds
from pyleaves.datasets import leaves_dataset, fossil_dataset, pnas_dataset, base_dataset
import neptune
import arrow
from pyleaves.utils import ensure_dir_exists
from more_itertools import unzip

##########################################################################
##########################################################################

def image_reshape(x):
    return [
        tf.image.resize(x, (7, 7)),
        tf.image.resize(x, (14, 14)),
        x
    ]


def load_img(image_path):#, img_size=(224,224)):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def resize(image, h=512, w=512):
    return tf.image.resize_with_pad(image, target_height=h, target_width=w)



def _cond_apply(x, y, func, prob):
    """Conditionally apply func to x and y with probability prob.
    """
    return tf.cond((tf.random.uniform([], 0, 1) >= (1.0 - prob)), lambda: func(x,y), lambda: (x,y))


def augment_sample(x, y):
    x = tf.image.random_flip_left_right(x, seed=2)
    x = tf.image.random_flip_up_down(x, seed=2)
    return x, y
                


def create_mnist_dataset(batch_size):
    (train_images, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
    train_images = train_images.reshape([-1, 28, 28, 1]).astype('float32')
    train_images = train_images/127.5  - 1
    dataset = tf.data.Dataset.from_tensor_slices(train_images)
    dataset = dataset.map(image_reshape)
    dataset = dataset.cache()
    dataset = dataset.shuffle(len(train_images))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(1)
    return dataset


def prep_dataset(dataset, batch_size=32, buffer_size=100, shuffle=False, target_size=(512,512), augment=False, aug_prob=1.0):
    dataset = dataset.map(lambda x,y: (resize(x, *target_size),y), num_parallel_calls=-1)
    dataset = dataset.cache()
    dataset = dataset.repeat()
    
    if augment:
        dataset = dataset.map(lambda x,y: _cond_apply(x, y, augment_sample, prob=aug_prob), num_parallel_calls=-1)
#         dataset = dataset.map(augment_sample, num_parallel_calls=-1)
    if shuffle:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(1)
    return dataset


def create_Imagenette_dataset(batch_size,
                              target_size=(512,512), 
                              augment_train=True,
                              aug_prob=1.0):
    
    data, info = tfds.load('Imagenette', as_supervised=True, with_info=True)
    train_data = prep_dataset(data['train'], 
                              batch_size=batch_size, 
                              buffer_size=info.splits['train'].num_examples,
                              shuffle=True,
                              target_size=target_size,
                              augment=augment_train,
                              aug_prob=aug_prob)
    val_data = prep_dataset(data['validation'], 
                            batch_size=batch_size,
                            target_size=target_size)
    
    return train_data, val_data, info



def partition_data(data, partitions=OrderedDict({'train':0.5,'test':0.5})):
    '''
    Split data into named partitions by fraction

    Example:
    --------
    >> split_data = partition_data(data, partitions=OrderedDict({'train':0.4,'val':0.1,'test':0.5}))
    '''
    num_rows = len(data)
    output={}
    taken = 0.0
    for k,v in partitions.items():
        idx = (int(taken*num_rows),int((taken+v)*num_rows))
        output.update({k:data[idx[0]:idx[1]]})
        taken+=v
    assert taken <= 1.0
    return output

def load_pnas_data(splits={'train':0.7,'validation':0.3}):
    data_files = pnas_dataset.PNASDataset()
    
    data_files.exclude_rare_classes(threshold=50)
    encoder = base_dataset.LabelEncoder(data_files.classes)
    data_files, _ = data_files.enforce_class_whitelist(class_names=encoder.classes)

    x = list(data_files.data['path'].values)#.reshape((-1,1))
    y = np.array(encoder.encode(data_files.data['family']))

    shuffled_data = list(zip(x,y))
    random.shuffle(shuffled_data)
    partitioned_data = partition_data(data=shuffled_data,
                                      partitions=OrderedDict(splits))
    split_data = {k:v for k,v in partitioned_data.items() if len(v)>0}
    
    for subset, subset_data in split_data.items():
        split_data[subset] = [list(i) for i in unzip(subset_data)]
    
    paths = tf.data.Dataset.from_tensor_slices(split_data['train'][0])
    labels = tf.data.Dataset.from_tensor_slices(split_data['train'][1])
    train_data = tf.data.Dataset.zip((paths, labels))
    train_data = train_data.map(lambda x,y: (tf.image.convert_image_dtype(load_img(x)*255.0,dtype=tf.uint8),y), num_parallel_calls=-1)
    
    paths = tf.data.Dataset.from_tensor_slices(split_data['validation'][0])
    labels = tf.data.Dataset.from_tensor_slices(split_data['validation'][1])
    validation_data = tf.data.Dataset.zip((paths, labels))
    validation_data = validation_data.map(lambda x,y: (tf.image.convert_image_dtype(load_img(x)*255.0,dtype=tf.uint8),y), num_parallel_calls=-1)
    
    return {'train':train_data, 
            'validation':validation_data}, data_files


def create_pnas_dataset(batch_size, 
                        target_size=(512,512),
                        splits={'train':0.7,'validation':0.3},
                        augment_train=True,
                        aug_prob=1.0):
    
    dataset, data_files = load_pnas_data(splits=splits)
    train_data = prep_dataset(dataset['train'], 
                              batch_size=batch_size, 
                              buffer_size=int(data_files.num_samples*splits['train']),
                              shuffle=True,
                              target_size=target_size,
                              augment=augment_train,
                              aug_prob=aug_prob)
    val_data = prep_dataset(dataset['validation'], 
                            batch_size=batch_size,
                            target_size=target_size)
    return train_data, val_data, data_files


# def create_pnas_dataset(batch_size):
#     data_files = pnas_dataset.PNASDataset()
#     dataset = tf.data.Dataset.from_tensor_slices(data_files.data['path'])
#     dataset = dataset.map(load_img, num_parallel_calls=-1)
#     dataset = dataset.map(resize, num_parallel_calls=-1)
#     dataset = dataset.cache()
#     dataset = dataset.shuffle(20)#len(data_files.data))
#     dataset = dataset.batch(batch_size, drop_remainder=True)
#     dataset = dataset.prefetch(1)
#     return dataset

##########################################################################
##########################################################################
    
preprocess_input(tf.zeros([4, 224, 224, 3]))
def apply_preprocess(x, y, num_classes=10):
    return preprocess_input(x), tf.one_hot(y, depth=num_classes)
    

def build_head(base, num_classes=10):
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    dense1 = tf.keras.layers.Dense(2048,activation='relu',name='dense1')#, kernel_initializer=tf.initializers.GlorotNormal())
    dense2 = tf.keras.layers.Dense(512,activation='relu',name='dense2')#, kernel_initializer=tf.initializers.GlorotNormal())
    prediction_layer = tf.keras.layers.Dense(num_classes,activation='softmax')#, kernel_initializer=tf.initializers.GlorotNormal())
    model = tf.keras.Sequential([
        base,
        global_average_layer,dense1,dense2,
        prediction_layer
        ])
    return model


def log_data(logs):
    for k, v in logs.items():
        neptune.log_metric(k, v)

        
##########################################################################
##########################################################################

        
def plot_sample(sample, num_res=1):
    num_samples = min(64, len(sample[0]))

    grid = gridspec.GridSpec(num_res, num_samples)
    grid.update(left=0, bottom=0, top=1, right=1, wspace=0.01, hspace=0.01)
    fig = plt.figure(figsize=[num_samples, num_res])
    for x in range(num_res):
        images = sample[x].numpy() #this converts the tensor to a numpy array
        images = np.squeeze(images)
        for y in range(num_samples):
            ax = fig.add_subplot(grid[x, y])
            ax.set_axis_off()
            ax.imshow((images[y] + 1.0)/2, cmap='gray')
    plt.show()
    

# neptune_logger = tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: log_data(logs))
neptune_logger = tf.keras.callbacks.LambdaCallback(on_batch_end=lambda batch, logs: log_data(logs),
                                                  on_epoch_end=lambda epoch, logs: log_data(logs))

In [9]:
from pyleaves.utils.neptune_utils import ImageLoggerCallback

def train_imagenette(PARAMS):

    neptune.append_tag(PARAMS['dataset_name'])
    neptune.append_tag(PARAMS['model_name'])
    
    K.clear_session()
    tf.random.set_seed(34)
    target_size = PARAMS['target_size']
    BATCH_SIZE = PARAMS['BATCH_SIZE']
    
    train_dataset, validation_dataset, info = create_Imagenette_dataset(BATCH_SIZE, target_size=target_size, augment_train=PARAMS['augment_train'])
    num_classes = info.features['label'].num_classes
    
    encoder = base_dataset.LabelEncoder(info.features['label'].names)
    
    train_dataset = train_dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)
    validation_dataset = validation_dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)
    
    PARAMS['num_classes'] = num_classes
    steps_per_epoch = info.splits['train'].num_examples//BATCH_SIZE
    validation_steps = info.splits['validation'].num_examples//BATCH_SIZE

    neptune.set_property('num_classes',num_classes)
    neptune.set_property('steps_per_epoch',steps_per_epoch)
    neptune.set_property('validation_steps',validation_steps)


    optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['learning_rate'])
    loss = 'categorical_crossentropy'
    METRICS = ['accuracy']

    base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                             include_top=False,
                                             input_tensor=Input(shape=(*target_size,3)))

    # TODO try freezing weights for input_shape != (224,224)

    model = build_head(base, num_classes=num_classes)

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=METRICS)

    callbacks = [neptune_logger,
                 ImageLoggerCallback(data=train_dataset, freq=10, max_images=-1, name='train', encoder=encoder),
                 ImageLoggerCallback(data=validation_dataset, freq=10, max_images=-1, name='val', encoder=encoder),
                 EarlyStopping(monitor='val_loss', patience=2, verbose=1)]

    model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
    pprint(PARAMS)
    history = model.fit(train_dataset,
                        epochs=10,
                        callbacks=callbacks,
                        validation_data=validation_dataset,
                        shuffle=True,
                        initial_epoch=0,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps)
    
    
def train_pnas(PARAMS):
    ensure_dir_exists(PARAMS['log_dir'])
    neptune.append_tag(PARAMS['dataset_name'])
    neptune.append_tag(PARAMS['model_name'])
    
    K.clear_session()
    tf.random.set_seed(34)
    
    splits = PARAMS['splits']
    train_dataset, validation_dataset, data_files = create_pnas_dataset(batch_size=PARAMS['BATCH_SIZE'],
                                                                        target_size=PARAMS['target_size'],
                                                                        splits=splits,
                                                                        augment_train=PARAMS['augment_train'])
    
    PARAMS['num_classes'] = data_files.num_classes #info.features['label'].num_classes
    PARAMS['splits_size'] = {'train':{},
                       'validation':{}}
    PARAMS['splits_size']['train'] = data_files.num_samples*PARAMS['splits']['train']
    PARAMS['splits_size']['validation'] = data_files.num_samples*PARAMS['splits']['validation']


    steps_per_epoch = PARAMS['splits_size']['train']//PARAMS['BATCH_SIZE']
    validation_steps = PARAMS['splits_size']['validation']//PARAMS['BATCH_SIZE']

    
#     num_classes = info.num_classes
#     num_samples = info.num_samples
    
    encoder = base_dataset.LabelEncoder(data_files.classes)
    
    train_dataset = train_dataset.map(lambda x,y: apply_preprocess(x,y,PARAMS['num_classes']),num_parallel_calls=-1)
    validation_dataset = validation_dataset.map(lambda x,y: apply_preprocess(x,y,PARAMS['num_classes']),num_parallel_calls=-1)
    
#     PARAMS['num_classes'] = num_classes
#     steps_per_epoch = int(num_samples*splits['train'])//PARAMS['BATCH_SIZE']
#     validation_steps = int(num_samples*splits['validation'])//PARAMS['BATCH_SIZE']

    neptune.set_property('num_classes',PARAMS['num_classes'])
    neptune.set_property('steps_per_epoch',steps_per_epoch)
    neptune.set_property('validation_steps',validation_steps)


    optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['lr'])

    base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                             include_top=False,
                                             input_tensor=Input(shape=(*PARAMS['target_size'],3)))

    model = build_head(base, num_classes=PARAMS['num_classes'])

    METRICS = ['accuracy']
    model.compile(optimizer=PARAMS['optimizer'],
                  loss=PARAMS['loss'],
                  metrics=METRICS)
    

    callbacks = [neptune_logger,
                 ImageLoggerCallback(data=train_dataset, freq=10, max_images=-1, name='train', encoder=encoder),
                 ImageLoggerCallback(data=validation_dataset, freq=10, max_images=-1, name='val', encoder=encoder),
                 EarlyStopping(monitor='val_loss', patience=25, verbose=1)]

    model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
    pprint(PARAMS)
    history = model.fit(train_dataset,
                        epochs=PARAMS['num_epochs'],
                        callbacks=callbacks,
                        validation_data=validation_dataset,
                        shuffle=True,
                        initial_epoch=0,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps)

In [3]:
# PARAMS = {'neptune_project_name':'jacobarose/sandbox',
#           'experiment_name':'imagenette_test',
#           'experiment_dir':'/media/data/jacob/sandbox_logs',
#           'experiment_start_time':arrow.utcnow().format('YYYY-MM-DD_HH-mm-ss'),
#           'optimizer':'Adam',
#           'loss':'categorical_crossentropy',
#           'lr':1e-6,
#           'target_size':(224,224), #(256,256),
#           'BATCH_SIZE':48,
#           'num_epochs':150}

# PARAMS['log_dir'] = os.path.join(PARAMS['experiment_dir'], PARAMS['experiment_name'], 'log_dir', PARAMS['loss'], PARAMS['experiment_start_time'])
# neptune.init(project_qualified_name=PARAMS['neptune_project_name'])
# neptune_tb.integrate_with_tensorflow()


# PARAMS['target_size'] = (256,256)
# PARAMS['BATCH_SIZE'] = 16

In [4]:
# train_dataset, validation_dataset, info = create_Imagenette_dataset(16, target_size=(224,224), augment_train=True)
# a = next(iter(train_data))
# print(a[0].numpy().min(),a[0].numpy().max())

# train_data, validation_data, info = create_pnas_dataset(batch_size=16,
#                                                   target_size=(224,224),
#                                                   splits={'train':0.7,'validation':0.3}, 
#                                                   augment_train=True)
# a = next(iter(train_data))
# print(a[0].numpy().min(),a[0].numpy().max())
# a[0].shape

In [5]:
# ensure_dir_exists(PARAMS['log_dir'])

# train_dataset, validation_dataset, info = create_Imagenette_dataset(PARAMS['BATCH_SIZE'], 
#                                                                     target_size=PARAMS['target_size'])
# PARAMS['num_classes'] = info.features['label'].num_classes
# PARAMS['splits_size'] = {'train':{},
#                    'validation':{}}
# PARAMS['splits_size']['train'] = info.splits['train'].num_examples
# PARAMS['splits_size']['validation'] = info.splits['validation'].num_examples


# steps_per_epoch = PARAMS['splits_size']['train']//PARAMS['BATCH_SIZE']
# validation_steps = PARAMS['splits_size']['validation']//PARAMS['BATCH_SIZE']
# # steps_per_epoch = info.splits['train'].num_examples//PARAMS['BATCH_SIZE']
# # validation_steps = info.splits['validation'].num_examples//PARAMS['BATCH_SIZE']

# optimizer = tf.keras.optimizers.Adam(learning_rate=PARAMS['lr'])

# big_base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
#                                              include_top=False,
#                                              input_tensor=Input(shape=(*PARAMS['target_size'],3)))

# big_model = build_head(big_base, num_classes=PARAMS['num_classes'])


# METRICS = ['accuracy']
# big_model.compile(optimizer=PARAMS['optimizer'],
#               loss=PARAMS['loss'],
#               metrics=METRICS)

# big_base.summary()

In [6]:
# target_size = (224,224)

# PARAMS['target_size'] = target_size
# with neptune.create_experiment(name=experiment_name+'-'+str(target_size), params=PARAMS):

#     train_dataset, validation_dataset, info = create_Imagenette_dataset(BATCH_SIZE, target_size=target_size)
#     num_classes = info.features['label'].num_classes

#     train_dataset = train_dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)
#     validation_dataset = validation_dataset.map(lambda x,y: apply_preprocess(x,y,num_classes),num_parallel_calls=-1)


#     steps_per_epoch = info.splits['train'].num_examples//BATCH_SIZE
#     validation_steps = info.splits['validation'].num_examples//BATCH_SIZE

#     loss = 'categorical_crossentropy'
#     METRICS = ['accuracy']

#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)

#     base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
#                                                     include_top=False,
#                                                     input_tensor=Input(shape=(*target_size,3)))

#     model = build_head(base, num_classes=num_classes)

#     model.compile(optimizer=optimizer,
#                   loss=loss,
#                   metrics=METRICS)

#     callbacks = [neptune_logger,
#                  EarlyStopping(monitor='val_loss', patience=5, verbose=1)]

    
#     model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))
    
#     history = model.fit(train_dataset,
#                         epochs=10,
#                         callbacks=callbacks,
#                         validation_data=validation_dataset,
#                         shuffle=True,
#                         initial_epoch=0,
#                         steps_per_epoch=steps_per_epoch,
#                         validation_steps=validation_steps)

In [7]:
# K.clear_session()

# big_target_size = (256,256)
# PARAMS['target_size'] = big_target_size
# with neptune.create_experiment(name=experiment_name+'-'+str(big_target_size), params=PARAMS):
#     BATCH_SIZE = 16
#     train_dataset, validation_dataset, info = create_Imagenette_dataset(BATCH_SIZE, target_size=big_target_size)
#     num_classes = info.features['label'].num_classes
    

#     steps_per_epoch = info.splits['train'].num_examples//BATCH_SIZE
#     validation_steps = info.splits['validation'].num_examples//BATCH_SIZE

#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)

#     big_base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
#                                                  include_top=False,
#                                                  input_tensor=Input(shape=(*big_target_size,3)))

#     big_model = build_head(big_base, num_classes=num_classes)

#     big_model.compile(optimizer=optimizer,
#                   loss=loss,
#                   metrics=METRICS)

#     callbacks = [neptune_logger,
#                  EarlyStopping(monitor='val_loss', patience=2, verbose=1)]

#     history = big_model.fit(train_dataset,
#                         epochs=10,
#                         callbacks=callbacks,
#                         validation_data=validation_dataset,
#                         shuffle=True,
#                         initial_epoch=0,
#                         steps_per_epoch=steps_per_epoch,
#                         validation_steps=validation_steps)

In [ ]:
# PARAMS = {'neptune_project_name':'jacobarose/sandbox',
#           'experiment_name':'pnas_test',
#           'loss':'Adam'}

PARAMS = {'neptune_project_name':'jacobarose/sandbox',
          'experiment_name':'pnas_test',
          'experiment_dir':'/media/data/jacob/sandbox_logs',
          'experiment_start_time':arrow.utcnow().format('YYYY-MM-DD_HH-mm-ss'),
          'optimizer':'Adam',
          'loss':'categorical_crossentropy',
          'lr':1e-6,
          'target_size':(224,224), #(256,256),
          'BATCH_SIZE':48,
          'num_epochs':150,
          'dataset_name':'pnas',
          'model_name':'vgg16',
          'augment_train':True,
          'splits':{'train':0.5,'validation':0.5}}

PARAMS['log_dir'] = os.path.join(PARAMS['experiment_dir'], PARAMS['experiment_name'], 'log_dir', PARAMS['loss'], PARAMS['experiment_start_time'])


# PARAMS['learning_rate'] = 1e-6
# PARAMS['target_size'] = (224,224)
# PARAMS['BATCH_SIZE'] = 48
# PARAMS['num_epochs'] = 150
# PARAMS['dataset_name'] = 'pnas'
# PARAMS['model_name'] = 'vgg16'
# PARAMS['augment_train'] = True
# PARAMS['splits'] = {'train':0.5,'validation':0.5}

neptune.init(project_qualified_name=PARAMS['neptune_project_name'])
with neptune.create_experiment(name=PARAMS['experiment_name']+'-'+str(PARAMS['splits']), params=PARAMS):
    train_pnas(PARAMS)

https://ui.neptune.ai/jacobarose/sandbox/e/SAN-434
{'BATCH_SIZE': 48,
 'augment_train': True,
 'dataset_name': 'pnas',
 'experiment_dir': '/media/data/jacob/sandbox_logs',
 'experiment_name': 'pnas_test',
 'experiment_start_time': '2020-07-01_05-58-58',
 'log_dir': '/media/data/jacob/sandbox_logs/pnas_test/log_dir/categorical_crossentropy/2020-07-01_05-58-58',
 'loss': 'categorical_crossentropy',
 'lr': 1e-06,
 'model_name': 'vgg16',
 'neptune_project_name': 'jacobarose/sandbox',
 'num_classes': 19,
 'num_epochs': 150,
 'optimizer': 'Adam',
 'splits': {'train': 0.5, 'validation': 0.5},
 'splits_size': {'train': 2657.0, 'validation': 2657.0},
 'target_size': (224, 224)}
Train for 55.0 steps, validate for 55.0 steps
Epoch 1/150
Batch 1: Logged 48 train images to neptune
Batch 1: Logged 48 val images to neptune
10/55 [====>.........................] - ETA: 3:41 - loss: 11.1833 - accuracy: 0.0812Batch 11: Logged 48 train images to neptune
Batch 11: Logged 48 val images to neptune
20/55 [==

In [ ]:
PARAMS = {'neptune_project_name':'jacobarose/sandbox',
          'experiment_name':'imagenette_test',
          'loss':'Adam'}


PARAMS['learning_rate'] = 1e-6
PARAMS['target_size'] = (224,224)
PARAMS['BATCH_SIZE'] = 16
PARAMS['dataset_name'] = 'imagenette'
PARAMS['model_name'] = 'vgg16'
PARAMS['augment_train'] = True
PARAMS['splits'] = {'train':0.7,'validation':0.3}

neptune.init(project_qualified_name=PARAMS['neptune_project_name'])
with neptune.create_experiment(name=PARAMS['experiment_name']+'-'+str(PARAMS['target_size']), params=PARAMS):
    train_imagenette(PARAMS)

In [ ]:
PARAMS['learning_rate'] = 1e-6
PARAMS['target_size'] = (256,256)
PARAMS['BATCH_SIZE'] = 16
with neptune.create_experiment(name=PARAMS['experiment_name']+'-'+str(PARAMS['target_size']), params=PARAMS):
    train_imagenette(PARAMS)

In [ ]:
PARAMS['learning_rate'] = 1e-6
PARAMS['target_size'] = (384,384)
PARAMS['BATCH_SIZE'] = 16
with neptune.create_experiment(name=experiment_name+'-'+str(PARAMS['target_size']), params=PARAMS):
    train_imagenette(PARAMS)

## Train Imagenette

In [ ]:
# train_dataset, validation_dataset = create_Imagenette_dataset(BATCH_SIZE)
# for sample in train_dataset.take(5): # the dataset has to fit in memory with eager iteration
#     print(sample[0].shape)
#     plot_sample([sample[0]/127.5-1.0], num_res=1)

# print([s.shape for s in sample])

In [ ]:
# sample[0].numpy().min()
# sample[0].numpy().max()

# print(preprocess_input(sample[0]).numpy().min())
# print(preprocess_input(sample[0]).numpy().max())

# dataset =  create_mnist_dataset(BATCH_SIZE)
# for sample in dataset.take(1): # the dataset has to fit in memory with eager iteration
#     plot_sample(sample, num_res=3)

# print([s.shape for s in sample])



# big_base.summary()

# base.summary()

# from tensorflow.keras.utils import plot_model
# plot_model(big_base, to_file='model.png', show_shapes=True)



# num_params_big = 0
# for layer in big_base.layers:
#     num_params_big += np.prod(layer.output_shape[1:])
# print(num_params_big)

# num_params = 0
# for layer in base.layers:
#     num_params += np.prod(layer.output_shape[1:])
# print(num_params)

# print(num_params_big - num_params)